### Setup

In [2]:
# this first setup block will pretty much always stay the same
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_multiple_reports, do_fit_scenarios
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [3]:
# set up the output directory for this Jupyter notebook
outdir = setup("20220815_scenario_projections.ipynb")

2022/08/02 08:14:32 INFO|============================================================


### Fit an initial scenario through Feb 2022

In [4]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20220815_aug_gov_briefing/20220815_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20220815_aug_gov_briefing/vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2022-10-18',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/08/01 14:14:30 INFO|-------------------------|{} Retrieving vaccinations data
2022/08/01 14:14:37 INFO|-------------------------|{} Constructing vaccination projections
2022/08/01 14:14:37 INFO|-------------------------|{} Retrieving hospitalizations data
2022/08/01 14:14:38 INFO|----------------------|{} Prepping Model
2022/08/01 14:14:38 INFO|-----------------------|{} Prepping Model
2022/08/01 14:15:32 INFO|----------------------|{} Model prepped for fitting in 54.13200379099999 seconds.
2022/08/01 14:15:32 INFO|----------------------|{} Will fit 25 times
2022/08/01 14:17:02 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 89.500094709 seconds: {0: {'co': 0.8773342920133839}, 14: {'co': 0.6854869128115542}, 28: {'co': 0.5378778726564099}, 42: {'co': 0.0005295181905471819}, 56: {'co': 0.6816113314978964}, 70: {'co': 0.8911952767526848}}
2022/08/01 14:17:02 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/08/01 14

2022/08/01 14:22:22 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 15.03286987499996 seconds: {504: {'co': 0.7977162242700062}, 518: {'co': 0.7923907094046225}, 532: {'co': 0.771324650296121}, 546: {'co': 0.7261267680111922}, 560: {'co': 0.7583946257234445}, 574: {'co': 0.7537571465068}}
2022/08/01 14:22:22 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/08/01 14:22:41 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 16.925778165999986 seconds: {532: {'co': 0.7711685074647038}, 546: {'co': 0.7277849260555459}, 560: {'co': 0.7548322212055819}, 574: {'co': 0.7638947932868047}, 588: {'co': 0.7998401988726278}, 602: {'co': 0.7810998735125031}}
2022/08/01 14:22:41 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/08/01 14:22:53 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 11.281610582999974 seconds: {560

### Create and run scenarios from Feb 2022 to present

In [4]:
multiprocess = 6

scenario_params = json.load(open("covid_model/analysis/20220815_aug_gov_briefing/20220815_scenario_params.json"))

model_args = {
    'base_spec_id': 3470 #model.spec_id # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# compute vacc effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for hosp_frac in [0.7, 0.8, 0.9]:
    for school_contact_increase in [1.1, 1.2]:
        for beta_mult in [0.95, 1.0, 1.05]:
            hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*hosp_frac), "2022-03-15": (0.34 + 0.66*hosp_frac), "2022-03-30": hosp_frac}
            beta_param_adjustment = [{"param": "betta",
                                      "attrs": {"variant": "ba45"},
                                      "mults": {"2020-01-01": beta_mult},
                                      "desc": "sensitivity analysis for beta"}]
            school_contact = [{"param": "betta",
                               "attrs": {'age': '0-19'},
                               "mults": {"2020-01-01": 1, "2022-08-15": school_contact_increase},
                               "desc": "After August 15th, beta increases by a factor of the school contact increase"}]
            lt5_vacc_adjust = [{"param": "immunity",
                                "attrs": {'age': '0-19', 'vacc': 'shot1'},
                                "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5,
                                          "2022-06-30": 0.98 + 0.02*vacc_eff_lt5, "2022-07-10": 0.97 + 0.03*vacc_eff_lt5,
                                          "2022-07-20": 0.96 + 0.04*vacc_eff_lt5, "2022-07-30": 0.95 + 0.05*vacc_eff_lt5},
                                "desc": "Starting June 20th, the share of 0-19 getting vaccinated who are < 5 are weighted by the vaccine effectiveness multiplier for this age group"}]
            scenario_model_args.append({'params_defs': scenario_params + beta_param_adjustment + school_contact + lt5_vacc_adjust,
                                        'hosp_reporting_frac': hrf,
                                        'tags': {'beta_mult': beta_mult, 'hosp_frac': hosp_frac, 'school_contact_increase': school_contact_increase}})
            
            

In [5]:
# check how many scenarios there are
len(scenario_model_args)

18

In [12]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/08/02 09:59:33 INFO|============================================================
2022/08/02 09:59:33 INFO|============================================================
2022/08/02 09:59:33 INFO|============================================================
2022/08/02 09:59:33 INFO|============================================================
2022/08/02 09:59:33 INFO|============================================================
2022/08/02 09:59:33 INFO|============================================================
2022/08/02 09:59:36 INFO|-----------|{'beta_mult': 0.95, 'hosp_frac': 0.7, 'school_contact_increase': 1.2} Retrieving vaccinations data
2022/08/02 09:59:36 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.1} Retrieving vaccinations data
2022/08/02 09:59:36 INFO|-----------|{'beta_mult': 1.05, 'hosp_frac': 0.7, 'school_contact_increase': 1.1} Retrieving vaccinations data
2022/08/02 09:59:36 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'sc

2022/08/02 10:01:07 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '0'}: Transmission control fit 1/3 completed in 25.321762375000006 seconds: {767: {'co': 0.8850657751214427}, 781: {'co': 0.8862508621347298}, 795: {'co': 0.8696644124923341}, 809: {'co': 0.8637662843799861}, 823: {'co': 0.865376565033736}, 837: {'co': 0.8561551078838291}}
2022/08/02 10:01:07 INFO|---------|{'beta_mult': 1.05, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '0'}: Running forward sim
2022/08/02 10:01:11 INFO|--------|{'beta_mult': 0.95, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '0'}: Transmission control fit 1/3 completed in 28.485094292000014 seconds: {767: {'co': 0.8850652107747761}, 781: {'co': 0.8862526571689991}, 795: {'co': 0.8696662744029191}, 809: {'co': 0.8636318569532261}, 823: {'co': 0.8648706618329708}, 837: {'co': 0.8543065012239389}}
2022/08/02 10:01:11 INFO|---------|{'beta_mult': 0.95, 'hosp_frac': 0.7, 

2022/08/02 10:01:42 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '2', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342909396418}, 14: {'co': 0.6854869158846699}, 28: {'co': 0.5312085495123385}, 42: {'co': 9.17481924148225e-14}, 56: {'co': 0.7004798432020942}, 70: {'co': 0.8356322478699926}, 84: {'co': 0.8298061649141574}, 98: {'co': 0.8311910701540246}, 112: {'co': 0.8778136907198419}, 126: {'co': 0.8241104215652926}, 140: {'co': 0.7606632612324116}, 154: {'co': 0.6711880275449327}, 168: {'co': 0.843030767299243}, 182: {'co': 0.7826561834881246}, 196: {'co': 0.8191830554171712}, 210: {'co': 0.7726034283937625}, 224: {'co': 0.7713676651676141}, 238: {'co': 0.7185596197504118}, 252: {'co': 0.7222209123366246}, 266: {'co': 0.6848822753308852}, 280: {'co': 0.7199443377723465}, 294: {'co': 0.7575590111802091}, 308: {'co': 0.8226615739216321}, 322: {'co': 0.8113710429684717}, 336: {'co': 0.782596504972317}, 350: {'co': 0.7952543151621825

2022/08/02 10:01:54 INFO|============================================================
2022/08/02 10:01:55 INFO|============================================================
2022/08/02 10:01:55 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.8, 'school_contact_increase': 1.1} Retrieving vaccinations data
2022/08/02 10:01:56 INFO|============================================================
2022/08/02 10:01:56 INFO|-----------|{'beta_mult': 1.05, 'hosp_frac': 0.8, 'school_contact_increase': 1.1} Retrieving vaccinations data
2022/08/02 10:01:57 INFO|============================================================
2022/08/02 10:01:57 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.8, 'school_contact_increase': 1.1} Constructing vaccination projections
2022/08/02 10:01:57 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.8, 'school_contact_increase': 1.1} Retrieving hospitalizations data
2022/08/02 10:01:57 INFO|-----------|{'beta_mult': 0.95, 'hosp_frac': 0.8, 'school_contact_increase': 1

2022/08/02 10:03:27 INFO|--------|{'beta_mult': 0.95, 'hosp_frac': 0.8, 'school_contact_increase': 1.1, 'fit_batch': '1'}: Transmission control fit 2/3 completed in 14.672803415999994 seconds: {795: {'co': 0.870927338659487}, 809: {'co': 0.8653918750408864}, 823: {'co': 0.8642757606682694}, 837: {'co': 0.8623817332581285}, 851: {'co': 0.849913563795655}, 865: {'co': 0.8722873824726862}}
2022/08/02 10:03:27 INFO|---------|{'beta_mult': 0.95, 'hosp_frac': 0.8, 'school_contact_increase': 1.1, 'fit_batch': '1'}: Running forward sim
2022/08/02 10:03:37 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.8, 'school_contact_increase': 1.2, 'fit_batch': '0'}: Transmission control fit 1/3 completed in 20.203722417000023 seconds: {767: {'co': 0.8893776991360267}, 781: {'co': 0.8873042375673695}, 795: {'co': 0.8708161945485056}, 809: {'co': 0.8646866057952948}, 823: {'co': 0.8670797240439465}, 837: {'co': 0.8586800993309768}}
2022/08/02 10:03:37 INFO|---------|{'beta_mult': 1.05, 'hosp_frac': 0.8, '

2022/08/02 10:03:59 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.8, 'school_contact_increase': 1.1, 'fit_batch': '2'}: Transmission control fit 3/3 completed in 15.50863258299998 seconds: {823: {'co': 0.8647979394702558}, 837: {'co': 0.8645064653318149}, 851: {'co': 0.8549395460233145}, 865: {'co': 0.883964943108827}, 879: {'co': 0.8807881909916497}, 893: {'co': 0.8828477779295238}}
2022/08/02 10:03:59 INFO|---------|{'beta_mult': 1.05, 'hosp_frac': 0.8, 'school_contact_increase': 1.1, 'fit_batch': '2'}: Running forward sim
2022/08/02 10:03:59 INFO|--------|{'beta_mult': 0.95, 'hosp_frac': 0.8, 'school_contact_increase': 1.2, 'fit_batch': '2', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342909396418}, 14: {'co': 0.6854869158846699}, 28: {'co': 0.5312085495123385}, 42: {'co': 9.17481924148225e-14}, 56: {'co': 0.7004798432020942}, 70: {'co': 0.8356322478699926}, 84: {'co': 0.8298061649141574}, 98: {'co': 0.8311910701540246}, 112: {'co': 0.8778136907198419}, 126: {'co': 0.82411042

2022/08/02 10:04:13 INFO|============================================================
2022/08/02 10:04:13 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.8, 'school_contact_increase': 1.1, 'fit_batch': '2'}: Transmission control fit 3/3 completed in 15.136696708999978 seconds: {823: {'co': 0.864453815413521}, 837: {'co': 0.863093446527912}, 851: {'co': 0.8516313151907835}, 865: {'co': 0.8781756219219286}, 879: {'co': 0.8710534430708597}, 893: {'co': 0.8729534615504377}}
2022/08/02 10:04:13 INFO|---------|{'beta_mult': 1.0, 'hosp_frac': 0.8, 'school_contact_increase': 1.1, 'fit_batch': '2'}: Running forward sim
2022/08/02 10:04:14 INFO|-----------|{'beta_mult': 0.95, 'hosp_frac': 0.9, 'school_contact_increase': 1.2} Retrieving vaccinations data
2022/08/02 10:04:15 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.8, 'school_contact_increase': 1.1, 'fit_batch': '2', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342909396418}, 14: {'co': 0.6854869158846699}, 28: {'co': 0.5312085495123385}

2022/08/02 10:05:03 INFO|--------|{'beta_mult': 0.95, 'hosp_frac': 0.9, 'school_contact_increase': 1.1} Model solved in 2.5498088749999965 seconds.
2022/08/02 10:05:03 INFO|--------|{'beta_mult': 0.95, 'hosp_frac': 0.9, 'school_contact_increase': 1.1} Will fit 3 times
2022/08/02 10:05:14 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.1} Model prepped for fitting in 68.24986425000003 seconds.
2022/08/02 10:05:14 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.1} Solving Model ODE
2022/08/02 10:05:15 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.9, 'school_contact_increase': 1.1} Model prepped for fitting in 68.753616916 seconds.
2022/08/02 10:05:15 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.9, 'school_contact_increase': 1.1} Solving Model ODE
2022/08/02 10:05:17 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.1} Model solved in 2.5674043329999563 seconds.
2022/08/02 10:05:17 INFO|--

2022/08/02 10:06:09 INFO|--------|{'beta_mult': 0.95, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'fit_batch': '1'}: Transmission control fit 2/3 completed in 16.459144666000043 seconds: {795: {'co': 0.8722535930297105}, 809: {'co': 0.8659734338673176}, 823: {'co': 0.8657298141561555}, 837: {'co': 0.8642579706952973}, 851: {'co': 0.853098885845821}, 865: {'co': 0.8762271156772663}}
2022/08/02 10:06:09 INFO|---------|{'beta_mult': 0.95, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'fit_batch': '1'}: Running forward sim
2022/08/02 10:06:10 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'fit_batch': '2'}: Transmission control fit 3/3 completed in 15.486633041999994 seconds: {823: {'co': 0.8663135449575707}, 837: {'co': 0.8665940273045211}, 851: {'co': 0.8583471587800339}, 865: {'co': 0.8878171051166552}, 879: {'co': 0.8853548132350749}, 893: {'co': 0.8882901827754601}}
2022/08/02 10:06:10 INFO|---------|{'beta_mult': 1.05, 'hosp_frac': 0.9, 

2022/08/02 10:06:28 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'fit_batch': '2'}: Transmission control fit 3/3 completed in 14.796271999999988 seconds: {823: {'co': 0.8658969845855815}, 837: {'co': 0.8652667911723446}, 851: {'co': 0.854370130585244}, 865: {'co': 0.881590132684993}, 879: {'co': 0.8762688603891351}, 893: {'co': 0.87902811715523}}
2022/08/02 10:06:28 INFO|---------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'fit_batch': '2'}: Running forward sim
2022/08/02 10:06:29 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'fit_batch': '2', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342909396418}, 14: {'co': 0.6854869158846699}, 28: {'co': 0.5312085495123385}, 42: {'co': 9.17481924148225e-14}, 56: {'co': 0.7004798432020942}, 70: {'co': 0.8356322478699926}, 84: {'co': 0.8298061649141574}, 98: {'co': 0.8311910701540246}, 112: {'co': 0.8778136907198419}, 126: {'co': 0.8241104215652

### Run the report for each fit model

In [13]:
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba45'], from_date='2022-01-01')

2022/08/02 10:13:41 INFO|============================================================
2022/08/02 10:13:41 INFO|--------|Solving model
2022/08/02 10:13:42 INFO|============================================================
2022/08/02 10:13:42 INFO|--------|Solving model
2022/08/02 10:13:42 INFO|============================================================
2022/08/02 10:13:42 INFO|--------|Solving model
2022/08/02 10:13:43 INFO|============================================================
2022/08/02 10:13:43 INFO|--------|Solving model
2022/08/02 10:13:44 INFO|============================================================
2022/08/02 10:13:44 INFO|--------|Solving model
2022/08/02 10:13:44 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/08/02 10:13:44 INFO|============================================================
2022/08/02 10:13:44 INFO|--------|Solving model
2022/08/02 

In [14]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-10-15', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-10-15', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/08/02 10:19:31 INFO|Projecting
2022/08/02 10:19:31 INFO|
2022/08/02 10:19:40 INFO|{'beta_mult': 0.95, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '2', 'run_type': 'fit'}: Running forward sim
2022/08/02 10:19:41 INFO|
2022/08/02 10:19:51 INFO|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '2', 'run_type': 'fit'}: Running forward sim
2022/08/02 10:19:52 INFO|
2022/08/02 10:20:01 INFO|{'beta_mult': 1.05, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '2', 'run_type': 'fit'}: Running forward sim
2022/08/02 10:20:02 INFO|
2022/08/02 10:20:11 INFO|{'beta_mult': 0.95, 'hosp_frac': 0.7, 'school_contact_increase': 1.2, 'fit_batch': '2', 'run_type': 'fit'}: Running forward sim
2022/08/02 10:20:12 INFO|
2022/08/02 10:20:39 INFO|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.2, 'fit_batch': '2', 'run_type': 'fit'}: Running forward sim
2022/08/02 10:20:40 INFO|
2022/08/02 10:20:49 INFO|{'beta_mult': 1.05, 